In [1]:
from osgeo import ogr
from osgeo import gdal
import bisect
import random
from shapely.geometry import Polygon, Point
from shapely import wkt
from shapely.geometry import shape
from shapely import speedups
speedups.disable()
import math
import pandas as pd
import sys
import datetime
from pyproj import Proj, Transformer
import pyproj

import cdsapi
import xarray as xr
from urllib.request import urlopen

import matplotlib.pyplot as plt
import geopandas as gpd

from pyproj import Proj
from shapely.ops import transform
from shapely.geometry import shape

import requests

In [2]:
# Parametros de control para la generacion
numeroIncendios = 2
filename = 'Realista_'+str(numeroIncendios)+'_Incendios_3_de_3.txt'

activo_pr = 0.95
agrupado_pr = 1
preparacion_rec_aereos = [15*60,20*60] # Entre 15 a 20 min

In [3]:
utm = pyproj.CRS('EPSG:32719')
wgs84 = pyproj.CRS('EPSG:4326')
utm_32518 = pyproj.CRS('EPSG:32518')

project = pyproj.Transformer.from_crs(wgs84, utm, always_xy=True).transform
project_2 = pyproj.Transformer.from_crs(utm_32518, utm, always_xy=True).transform
project_3 = pyproj.Transformer.from_crs(utm_32518, wgs84, always_xy=True).transform
project_4 = pyproj.Transformer.from_crs(utm, wgs84, always_xy=True).transform
project_5 = pyproj.Transformer.from_crs(utm, utm_32518, always_xy=True).transform
project_6 = pyproj.Transformer.from_crs(wgs84, utm_32518, always_xy=True).transform

### Importar DataSet y crear instancias

In [4]:
DataSet = 'dataset.csv'
Recursos_aereos = 'recursos_aereos.csv'
Recursos_combate = 'recursos_combate.csv'

In [5]:
DataSetDF = pd.read_csv(DataSet)
DataSetDF.head(5)

,Unnamed: 0,pendiente,VPL,coords,WGS84(GPS),date,time,H (%),T (C),V (m/s)
0,0,6.66667,0.005429,"(781410.8857684857, 5882854.141089097)","(-71.83108423317948, -37.15838297117608)",2017-02-14,17:00,71.8215,17.91930,3.33461
1,1,13.33330,0.001672,"(670918.3050503387, 5841153.4465974495)","(-73.06484307489475, -37.56079948860041)",2015-12-30,20:00,64.6337,14.60420,3.22632
2,2,-43.33330,0.005973,"(715599.1743549515, 5727723.4737017)","(-72.52515166055817, -38.57280701983519)",2019-03-18,10:00,90.5010,2.85562,9.23614
3,3,37.77780,0.001325,"(705019.3597288682, 5658491.045153752)","(-72.62582955383753, -39.19866930544828)",2017-01-14,08:00,16.8809,13.16230,8.31740
4,4,-4.44440,0.009777,"(667973.0952557751, 5841184.090236984)","(-73.09817912445067, -37.56106520866647)",2018-01-07,21:00,25.1265,22.06340,4.96412


In [6]:
Recursos_combatedf = pd.read_csv(Recursos_combate,index_col=False)
Recursos_combatedf.head()

,Clave,Base,Subgerencia,Área Patrimonio,EESS,Geo Ref,Geográfica,Dotación,F. de Combate,UTM (X) epsg:32518,UTM (Y) epsg:32518,Geo Lat.,Geo Lon.,Región,Comuna,Tipo
0,315,Cañete,Norte,Cañete,Serfonac,HO 1542 D,Cañete,14,10,641026.36,5814827.40,"37º 48' 11""","73º 23' 53""",Biobio,Cañete,TERRESTRE
1,711,Cholchol,Sur,Carahue,Serfonac,IS 2392 C,4.26 km SE Chol Chol,14,10,691960.24,5722992.44,"38° 37' 14""","72° 47' 41""",Araucania,Cholchol,TERRESTRE
2,713,Cholchol,Sur,Carahue,Serfonac,IS 2392 C,4.26 km SE Chol Chol,6,4,691960.24,5722992.44,"38° 37' 14""","72° 47' 41""",Araucania,Cholchol,CISTERNA
3,414,Cholguan,Norte,Los Angeles,Serfonac,GP 8360 C,6.4 km SW Yungay,14,10,759953.34,5882654.51,"37º 09' 59""","72º 04' 20""",Ñuble,Yungay,TERRESTRE
4,316,Contulmo,Norte,Cañete,Serfonac,HY 9156 A,Contulmo,14,10,656287.37,5791518.34,"38 °00' 37""","73° 13' 10""",Biobio,Contulmo,TERRESTRE


In [7]:
Recursos_aereosdf = pd.read_csv(Recursos_aereos,index_col=False)
Recursos_aereosdf.head(5)

,Clave,Base,Área Patrimonio,EESS,Geo Ref,Geográfica,UTM (X) epsg:32518,UTM (Y) epsg:32518,Geo Lat.,Geo Lon.,Modelo Aeronave,Velocidad Crucero (Km/h),Autonomía de Vuelo,Capacidad de Bombardeo (lt.),Región,Comuna,Tipo
0,Z-5,Angol,Nacimiento,Cerda,HR 1504 C,Aeródromo Angol,703678.66,5814254.95,"37º 47' 46""","72º 41' 11""",Air Tractor 802,259,NaN,3000,IX- Araucanía,Angol,CISTERNAAEREA
1,Z-6,Angol,Nacimiento,Cerda,HR 1504 C,Aeródromo Angol,703678.66,5814254.95,"37º 47' 46""","72º 41' 11""",Air Tractor 802,259,NaN,3000,IX- Araucanía,Angol,CISTERNAAEREA
2,Z-9,Angol,Nacimiento,Cerda,HR 1504 C,Aeródromo Angol,703678.66,5814254.95,"37º 47' 46""","72º 41' 11""",Air Tractor 802,259,NaN,3000,IX- Araucanía,Angol,CISTERNAAEREA
3,Z-4,Cauquenes,Talca,Indef,DW 2742 C,9.5 Km N de Cauquenes,741464.60,6026371.60,"35º 52' 37""","72º 19' 31""",Air Tractor 802,259,NaN,3000,VII- Maule,Cauquenes,CISTERNAAEREA
4,Z-3,María Dolores,Los Angeles,Serfonac,GX 5928 B,Aeródromo Los Angeles,728021.07,5858347.74,"37º 23' 36""","72º 25' 27""",Air Tractor 802,259,NaN,3000,VIII- Biobio,Los Ángeles,CISTERNAAEREA


In [8]:
# Pre-proceso de los recursos antes de crear intancias
# buscar unicos de base
# por base ver si existe la misma cantidad de recursos en cisterna y helicombate si hay más 
# en en recursos aereos

recursosDF = pd.DataFrame(columns=['Clave','UTM (X)','UTM (Y)','Tipo'])
bases = set(Recursos_combatedf.Base.unique().tolist()) | set(Recursos_aereosdf.Base.unique().tolist())

for base in bases:
    combat = Recursos_combatedf[Recursos_combatedf.Base==base]
    aereos = Recursos_aereosdf[Recursos_aereosdf.Base==base]
    #Cisterna
    if len(combat[Recursos_combatedf.Tipo=='CISTERNA']) < len(aereos[Recursos_aereosdf.Tipo=='CISTERNAAEREA']):
        dif = len(aereos[Recursos_aereosdf.Tipo=='CISTERNAAEREA']) - len(combat[Recursos_combatedf.Tipo=='CISTERNA'])
        for i in aereos[Recursos_aereosdf.Tipo=='CISTERNAAEREA'][['Clave','UTM (X) epsg:32518','UTM (Y) epsg:32518','Tipo']].values.tolist()[0:dif]:
            recursosDF.loc[len(recursosDF.index)] = i
    #Helicombate
    if len(combat[Recursos_combatedf.Tipo=='HELICOMBATE']) < len(aereos[Recursos_aereosdf.Tipo=='HELICOMBATE']):
        dif = len(aereos[Recursos_aereosdf.Tipo=='HELICOMBATE']) - len(combat[Recursos_combatedf.Tipo=='HELICOMBATE'])
        for i in aereos[Recursos_aereosdf.Tipo=='HELICOMBATE'][['Clave','UTM (X) epsg:32518','UTM (Y) epsg:32518','Tipo']].values.tolist()[0:dif]:
            recursosDF.loc[len(recursosDF.index)] = [*i[0:3],'CHINOOK']
    #Todos
    #print(combat[['Clave','UTM (X) epsg:32518','UTM (Y) epsg:32518','Tipo']])
    insertar = combat[['Clave','UTM (X) epsg:32518','UTM (Y) epsg:32518','Tipo']].copy()
    insertar.rename(columns = {'UTM (X) epsg:32518':'UTM (X)','UTM (Y) epsg:32518':'UTM (Y)'}, inplace = True)
    recursosDF = recursosDF.append(insertar, ignore_index = True)

del combat, aereos, insertar

<ipython-input-8-cf1c5bd4a26b>:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if len(combat[Recursos_combatedf.Tipo=='CISTERNA']) < len(aereos[Recursos_aereosdf.Tipo=='CISTERNAAEREA']):
<ipython-input-8-cf1c5bd4a26b>:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if len(combat[Recursos_combatedf.Tipo=='HELICOMBATE']) < len(aereos[Recursos_aereosdf.Tipo=='HELICOMBATE']):
<ipython-input-8-cf1c5bd4a26b>:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dif = len(aereos[Recursos_aereosdf.Tipo=='CISTERNAAEREA']) - len(combat[Recursos_combatedf.Tipo=='CISTERNA'])
<ipython-input-8-cf1c5bd4a26b>:15: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  for i in aereos[Recursos_aereosdf.Tipo=='CISTERNAAEREA'][['Clave','UTM (X) epsg:32518','UTM (Y) epsg:32518','Tipo']].values.tolist()[0:dif]:
<ipython-input-8-cf1c5bd4a26b>:19: UserWarning: Boolean Series key will be reind

In [9]:
recursosDF

,Clave,UTM (X),UTM (Y),Tipo
0,416,711342.26,5857645.51,TERRESTRE
1,414,759953.34,5882654.51,TERRESTRE
2,315,641026.36,5814827.40,TERRESTRE
3,Z-1,729833.95,5887726.81,CISTERNAAEREA
4,W-9,729756.84,5886710.96,CHINOOK
5,412,729756.84,5886710.96,HELITRANSPORTADA
6,411,729756.84,5886710.96,CISTERNA
7,419,729756.84,5886710.96,MECANIZADA
8,Z-3,728021.07,5858347.74,CISTERNAAEREA
9,314,728099.06,5858499.85,HELITRANSPORTADA


In [10]:
def distance(origin, destination):
    """
    Calculate the Haversine distance.

    Parameters
    ----------
    origin : tuple of float
        (lat, long)
    destination : tuple of float
        (lat, long)

    Returns
    -------
    distance_in_km : float

    Examples
    --------
    origin = (48.1372, 11.5756)  # Munich
    >>> destination = (52.5186, 13.4083)  # Berlin
    >>> round(distance(origin, destination), 1)
    504.2
    """
    lat1, lon1 = origin
    lat2, lon2 = destination
    radius = 6371  # km

    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = (math.sin(dlat / 2) * math.sin(dlat / 2) +
         math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) *
         math.sin(dlon / 2) * math.sin(dlon / 2))
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    d = radius * c

    return d

In [11]:
velocidades = {'HELITRANSPORTADA':203, 'CISTERNAAEREA':259, 'CHINOOK':222}

In [12]:
# Se excluyen DE VERDAD los interfaz
recursosDF = recursosDF[recursosDF["Tipo"]!="Interfaz"]

In [13]:
ids = DataSetDF[['date','time']]
unique = {}

for i, row in ids.iterrows():
    tupl = (row['date'],row['time'])
    if tupl not in unique:
        unique[tupl] = [i]
    else:
        unique[tupl].append(i)

counter = dict()
        
for ind in unique.values():
    size = len(ind)
    if size not in counter:
        counter[size] = [ind]
    else:
        counter[size].append(ind)
    
idxPorConcurrencia = [counter[val] for val in sorted(counter.keys())]


In [14]:
### Seleccion de los incendios
grupoMuestra = random.randint(numeroIncendios,len(idxPorConcurrencia)-1)
grupoDeFocos = random.randint(0,len(idxPorConcurrencia[grupoMuestra])-1)

mascara = [True]*numeroIncendios + [False]*(grupoMuestra+1-numeroIncendios)
random.shuffle(mascara)

idsIncendios = [item for keep, item in zip(mascara, idxPorConcurrencia[grupoMuestra][grupoDeFocos]) if keep]

incendios = DataSetDF.iloc[idsIncendios, :]
print(incendios)

      Unnamed: 0  pendiente       VPL                                  coords  \
5695        5695  -20.00000  0.001325  (762885.9452892106, 5781203.662854449)   
7652        7652    4.44444  0.005429    (705503.2766551942, 5843807.8481946)   

                                    WGS84(GPS)        date   time    H (%)  \
5695  (-72.00288528603744, -38.07875936752243)  2016-01-28  11:00  88.4426   
7652  (-72.6742819689354, -37.529828389996005)  2016-01-28  11:00  91.4564   

         T (C)  V (m/s)  
5695  10.93110  3.80625  
7652   9.91299  2.61924  


In [15]:
# Formateo de la info para generar los datos de salida
focos = list(incendios['WGS84(GPS)'])
timestampFocos = [date+' '+time for date,time in zip(incendios['date'],incendios['time'])]

print(focos)
print(timestampFocos)

['(-72.00288528603744, -38.07875936752243)', '(-72.6742819689354, -37.529828389996005)']
['2016-01-28 11:00', '2016-01-28 11:00']


In [16]:
# se excluyen a las interfaz
import numpy as np
import numpy.random as rn
from scipy.stats import truncexpon
from ast import literal_eval as make_tuple
import requests

from datetime import datetime, timedelta

# id >> tipo >> horasDeTrabajo >> latitud >> longitud >> estaActivo >> estaAgrupado;
# horasDeTrabajo -> 0-

recursos = [] # id, tipo, horasTrabajo, lat, lon, activo, agrupado

contador_diferencia = [] #un calculo rápido para la diferencia en distancias
velocidad = [] #un calculo rápido para la velocidad en ruta
conversion = 11.58 #m/s

#por foco
# focos =['(-72.95020583491079, -37.73029279769046)', '(-73.01273041486445, -38.16781738573914)', '(-73.32945304026725, -38.21793985248813)']
# timestampFocos = ['12/22/2015T8:00', '12/22/2015T8:00', '12/22/2015T8:00']

for index,item in recursosDF.iterrows():
    item['Clave']
    point_32518 = Point(item['UTM (X)'],item['UTM (Y)'])
    point_wgs = transform(project_3,point_32518) # de epsg:32518 a WGS84 epsg:4326
    activo = 1 if (rn.rand()<activo_pr) else 0
    horasDeTrabajo = truncexpon.rvs(8) if (activo) else 0.0
    asignado = -1
    agrupado = 1 if (rn.rand()<agrupado_pr) else 0
    etas = []
    for timestampFoco,foco in zip(timestampFocos, focos):
        foco_wgs = Point(*make_tuple(foco))
        foco_32518 = transform(project_6, foco_wgs)
        timestamp = datetime.strptime(timestampFoco,'%Y-%m-%d %H:%M')
        
        #api:url con epsg:4326
        url = 'http://router.project-osrm.org/route/v1/driving/'+str(point_wgs.x)+','\
                +str(point_wgs.y)+';'+str(foco_wgs.x)+','+str(foco_wgs.y)+'?overview=false'
        res = requests.get(url)
        if item['Tipo'] in velocidades:
            duracion = (distance((point_wgs.x,point_wgs.y), (foco_wgs.x,foco_wgs.y))/velocidades[item['Tipo']])*3600
            duracion += random.randint(*preparacion_rec_aereos)
        else:
            if res.status_code == 200:
                if res.json()['code']=='Ok':
                    distancia = res.json()['routes'][0]['distance'] #en metros
                    duracion = res.json()['routes'][0]['duration'] #en segundos
                    contador_diferencia.append(distancia-point_32518.distance(foco_32518))
                    velocidad.append(distancia/duracion) # en m/s
            else:
                print('Error de Request.')
                if velocidad:
                    conversion = np.mean(velocidad)
                duracion = point_32518.distance(foco_32518) / conversion
        timestamp = timestamp+timedelta(seconds=duracion)
        etas.append(timestamp.strftime("%Y-%m-%dT%H:%M"))
    #print([item['Clave'],item['Tipo'],horasDeTrabajo,point_wgs.x,point_wgs.y,activo,agrupado,*etas])
    recursos.append([item['Clave'],item['Tipo'],horasDeTrabajo,point_wgs.x,point_wgs.y,activo,asignado,agrupado,*etas])

del point_32518, point_wgs, horasDeTrabajo, activo, agrupado, res, url, foco_wgs, foco_32518

In [17]:
# LATITUD_FOCO_1 LONGITUD_FOCO_1
#  TIMESTAMP_INICIO VALOR_RODAL_USD_POR_HECTAREA MODELO_COMBUSTIBLE DISTANCIA_A_CIUDAD_MAS_CERCANA CANTIDAD_METROS_CONSTRUIDA
vpl_val = [0.01888,0.016027,0.010235,0.00869,0.001009,0.007603,0.008147,0.001672,0.004886,0.010321,0.009234,0.001787,0.004342,0.002249,0.001441,0.000979,0.001556,0.002365,0.013174,0.005973,0.002481,0.002712,0.006516,0.003255,0.002596,0.009777,0.005429,0.003799,0.001325,0.002134,0.001903]
vpl_key = ['PCH1' ,'PCH2'  ,'PCH3'  ,'PCH4' ,'PCH5'  ,'MT01'  ,'MT02'  ,'MT03'  ,'MT04'  ,'MT05'  ,'MT06'  ,'MT07'  ,'MT08'  ,'BN01'  ,'BN02'  ,'BN03'  ,'BN04'  ,'BN05'  ,'PL01'  ,'PL02'  ,'PL03'  ,'PL04'  ,'PL05'  ,'PL06'  ,'PL07'  ,'PL08'  ,'PL09'  ,'PL10'  ,'PL11'  ,'DX01'  ,'DX02']

modelo_mapa = {}
    
for val,key in zip(vpl_val,vpl_key):
    modelo_mapa[val] = key

def VPL_key(value):
    closer = value
    if value not in modelo_mapa:
        diff = math.inf
        
        for val in modelo_mapa:
            if math.fabs(value-val)<diff:
                closer = val
                diff = math.fabs(value-val)
    
    return modelo_mapa[closer]



def printIncendios(printer=print):
    printer(numeroIncendios)
    
    for i,incendio in incendios.iterrows():
        H = incendio['H (%)']
        V = incendio['V (m/s)']
        D = 0.0
        T = incendio['T (C)']
        P = incendio['pendiente']
        VPL = incendio['VPL']
        time_stamp = incendio['date']+"T"+incendio['time']
        rodal = 9000
        modelo_combustile = VPL_key(incendio['VPL'])
#         modelo_combustile = modelo_mapa[incendio['VPL']]
        distancia_ciudad = 50000
        metros_construido = 0
        printer("".join(incendio['WGS84(GPS)'][1:-1].split(','))) ### ODIO PANDAS
        printer(H, V, P, T, P, VPL, time_stamp, rodal, modelo_combustile, distancia_ciudad, metros_construido)
printIncendios()

2
-72.00288528603744 -38.07875936752243
88.4426 3.80625 -20.0 10.9311 -20.0 0.001325 2016-01-28T11:00 9000 PL11 50000 0
-72.6742819689354 -37.529828389996005
91.4564 2.61924 4.44444 9.91299 4.44444 0.005429 2016-01-28T11:00 9000 PL09 50000 0


In [18]:
def printRecursos(printer=print):
    printer(len(recursos))
    for r in recursos:
        string = str()
        for item in r:
            string += str(item) + ' '
        printer(string)

In [19]:
def printMatriz(printer=print):
    for incendio in range(len(focos)):
        for r in recursos:
            printer(r[0],incendio+1, 1)

In [20]:
str_rendimientos = '''176.6666667 176.6666667 176.6666667 530 441.6666667 441.6666667 441.6666667
176.6666667 176.6666667 176.6666667 530 441.6666667 441.6666667 441.6666667
176.6666667 176.6666667 176.6666667 530 441.6666667 441.6666667 441.6666667
176.6666667 176.6666667 176.6666667 530 441.6666667 441.6666667 441.6666667
176.6666667 176.6666667 176.6666667 530 441.6666667 441.6666667 441.6666667
176.6666667 176.6666667 176.6666667 530 441.6666667 441.6666667 441.6666667
176.6666667 176.6666667 176.6666667 530 441.6666667 441.6666667 441.6666667
176.6666667 176.6666667 176.6666667 530 441.6666667 441.6666667 441.6666667
176.6666667 176.6666667 176.6666667 530 441.6666667 441.6666667 441.6666667
176.6666667 176.6666667 176.6666667 530 441.6666667 441.6666667 441.6666667
176.6666667 176.6666667 176.6666667 530 441.6666667 441.6666667 441.6666667
176.6666667 176.6666667 176.6666667 530 441.6666667 441.6666667 441.6666667
176.6666667 176.6666667 176.6666667 530 441.6666667 441.6666667 441.6666667
176.6666667 176.6666667 176.6666667 530 441.6666667 441.6666667 441.6666667
176.6666667 176.6666667 176.6666667 530 441.6666667 441.6666667 441.6666667
176.6666667 176.6666667 176.6666667 530 441.6666667 441.6666667 441.6666667
176.6666667 176.6666667 176.6666667 530 441.6666667 441.6666667 441.6666667
176.6666667 176.6666667 176.6666667 530 441.6666667 441.6666667 441.6666667
176.6666667 176.6666667 176.6666667 530 441.6666667 441.6666667 441.6666667
176.6666667 176.6666667 176.6666667 530 441.6666667 441.6666667 441.6666667
176.6666667 176.6666667 176.6666667 530 441.6666667 441.6666667 441.6666667
176.6666667 176.6666667 176.6666667 530 441.6666667 441.6666667 441.6666667
176.6666667 176.6666667 176.6666667 530 441.6666667 441.6666667 441.6666667
176.6666667 176.6666667 176.6666667 530 441.6666667 441.6666667 441.6666667
176.6666667 176.6666667 176.6666667 530 441.6666667 441.6666667 441.6666667
176.6666667 176.6666667 176.6666667 530 441.6666667 441.6666667 441.6666667
176.6666667 176.6666667 176.6666667 530 441.6666667 441.6666667 441.6666667
176.6666667 176.6666667 176.6666667 530 441.6666667 441.6666667 441.6666667
176.6666667 176.6666667 176.6666667 530 441.6666667 441.6666667 441.6666667
176.6666667 176.6666667 176.6666667 530 441.6666667 441.6666667 441.6666667
176.6666667 176.6666667 176.6666667 530 441.6666667 441.6666667 441.6666667'''
def printRendimientos(printer=print):
    printer(str_rendimientos)

In [21]:
str_costos = '''4988 14
30 14
37 21
103 83
4204 4204
53663 53663
4204 4204'''
def printCostos(printer=print):
    printer(str_costos)

In [22]:
#Output practicamente listo. Solo Faltaria el timestamp del momento del llamado al solver
def outputData(printer=print):
    printer(incendios.iloc[0]['date']+'T'+incendios.iloc[0]['time'])
    printIncendios(printer)
    printRecursos(printer)
    printRendimientos(printer)
    printCostos(printer)
    printMatriz(printer)
    
def outputToFile(filename):
    with open(filename, 'w') as f:
        def wrapper(*argv):
            for arg in argv:
                f.write(str(arg)+' ')
            f.write('\n')
        
        outputData(wrapper)

In [23]:
outputToFile(filename)
# outputData()